### Importations et chargements

In [ ]:
## Management du drive
from google.colab import drive
import os
import glob
drive.mount('/content/drive')

## Importation de modules et librairies utiles
import numpy as np
import pandas as pd
from librosa.feature import mfcc, delta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from pathlib import Path
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

Mounted at /content/drive


## Modalité - Son

### Jeu de sons pour la classification de compositeur : maps_composers
Données : 120 enregistrements de musique de piano de 30s </br>
Labels : 4 compositeurs (Beethoven, Chopin, Liszt, Mozart)

In [ ]:
# Commandes et imports spécifiques
%cd /content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/maps_composers
from sam_io import read_wav

/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/maps_composers


##### Extraction des descipteurs

*(Si plus de temps, implémenter les alternatives suivantes :*
- *Calculer les descripteurs Chroma, ∆Chroma et ∆∆Chroma*
- *Aggréger les features en un histogramme plutôt qu'en moyenne)*

In [ ]:
# Créer une fonction qui lit chaque fichier son
def read_files(path):
  my_data = []
  compositeurs = [composer for composer in os.listdir(path) if not 'py' in composer and not '.csv' in composer]
  for compositeur in compositeurs:
    new_path = os.path.join(path, compositeur)
    enregistrements = [enregistrement for enregistrement in os.listdir(new_path)]
    for enregistrement in enregistrements:
      sound_file = os.path.join(new_path, enregistrement)
      fs, x = read_wav(sound_file) # fs = fréquence d'échantillonage, x = le singal numérique
      # Nota Bene : on ajoute pas la fréquence d'échantillonage dans notre tableau de donnée car c'est toujours la même :
      # Quelque soit le son, fs = 44100
      # (de quoi récupérer toute l'information nécessaire selon le critère de Shannon sur des sons audibles)
      my_data.append(np.array([compositeur, x], dtype=object)) # to support indexing python regular list of lists with array
  my_data = np.array(my_data)
  return my_data

In [ ]:
# Appelle la fonction et arrange un dataframe associé à nos données
path = "/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/maps_composers"
my_data = read_files(path)
df = pd.DataFrame(my_data)
df.columns = ["Compositeurs", "Signal numérique"]
df

,Compositeurs,Signal numérique
0,chopin,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,chopin,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,chopin,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,chopin,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,chopin,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
115,beethoven,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
116,beethoven,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
117,beethoven,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
118,beethoven,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# Prend les extraits des morceaux de 10 à 15s :
fs = 44100
for i,x in enumerate(df['Signal numérique']):
  df['Signal numérique'][i] = x[10*fs:15*fs] # La 10ième seconde, c'est le (10 * fs)ième echantillons, etc.
df

,Compositeurs,Signal numérique
0,chopin,"[0.0533447265625, 0.050689697265625, 0.0474853..."
1,chopin,"[-0.051910400390625, -0.05487060546875, -0.057..."
2,chopin,"[0.0262451171875, 0.025848388671875, 0.0253601..."
3,chopin,"[0.020904541015625, 0.032623291015625, 0.04464..."
4,chopin,"[-0.003143310546875, -0.0030517578125, -0.0028..."
...,...,...
115,beethoven,"[0.001953125, 0.001953125, 0.00189208984375, 0..."
116,beethoven,"[-0.01007080078125, -0.01324462890625, -0.0163..."
117,beethoven,"[-0.009490966796875, -0.0093994140625, -0.0092..."
118,beethoven,"[-0.001220703125, -0.001434326171875, -0.00158..."


In [ ]:
# Calcule le pas et la longueur de la fenêtre pour segmenter les sons (échantillonage régulier)
# par trames de 100 ms environ et ajouter 50% de superposition
long_trame = 0.1 # 100 ms = 0.1 s
long_fene = int(fs*long_trame)
pas = int(long_fene/2)

# Calcule les descripteurs MFCC, ∆MFCC et ∆∆MFCC sur chaque trame
MFCC = []
delta_MFCC = []
delta2_MFCC = []
for x in df['Signal numérique']:
  x_mfcc = mfcc(y=x, sr=fs, hop_length=pas, n_fft = long_fene)
  mfcc_delta = delta(x_mfcc)
  mfcc_delta2 = delta(x_mfcc, order=2)
  MFCC.append(x_mfcc)
  delta_MFCC.append(mfcc_delta)
  delta2_MFCC.append(mfcc_delta2)

In [ ]:
# Ajoute trois nouvelles colonne à notre dataframe des données
df['MFCC'] = MFCC
df['Delta MFCC'] = delta_MFCC
df['Delta2 MFCC'] = delta2_MFCC
df

,Compositeurs,Signal numérique,MFCC,Delta MFCC,Delta2 MFCC
0,chopin,"[0.0533447265625, 0.050689697265625, 0.0474853...","[[-312.45836542709327, -335.35174710281143, -3...","[[-5.208176414613265, -5.208176414613265, -5.2...","[[7.342120188324413, 7.342120188324413, 7.3421..."
1,chopin,"[-0.051910400390625, -0.05487060546875, -0.057...","[[-314.71879604850164, -355.6567374810345, -36...","[[-3.5721087631905304, -3.5721087631905304, -3...","[[4.204460150627886, 4.204460150627886, 4.2044..."
2,chopin,"[0.0262451171875, 0.025848388671875, 0.0253601...","[[-454.77363406651415, -531.7426883560626, -53...","[[7.715278236473325, 7.715278236473325, 7.7152...","[[0.15417098189954306, 0.15417098189954306, 0...."
3,chopin,"[0.020904541015625, 0.032623291015625, 0.04464...","[[-238.57030824828675, -295.3045373706068, -28...","[[-3.6395510972983307, -3.6395510972983307, -3...","[[1.5240815909817902, 1.5240815909817902, 1.52..."
4,chopin,"[-0.003143310546875, -0.0030517578125, -0.0028...","[[-447.4002429810664, -467.3060022084104, -471...","[[-3.876535774779619, -3.876535774779619, -3.8...","[[1.7742695968054312, 1.7742695968054312, 1.77..."
...,...,...,...,...,...
115,beethoven,"[0.001953125, 0.001953125, 0.00189208984375, 0...","[[-619.5332876158313, -658.623698584141, -668....","[[-13.011103235917975, -13.011103235917975, -1...","[[1.8538293961178276, 1.8538293961178276, 1.85..."
116,beethoven,"[-0.01007080078125, -0.01324462890625, -0.0163...","[[-300.03756716075947, -347.46476156711594, -3...","[[-17.766107173116826, -17.766107173116826, -1...","[[0.7849762270468268, 0.7849762270468268, 0.78..."
117,beethoven,"[-0.009490966796875, -0.0093994140625, -0.0092...","[[-470.533732973774, -501.03628535809986, -507...","[[-8.59627716650855, -8.59627716650855, -8.596...","[[1.7545349805609, 1.7545349805609, 1.75453498..."
118,beethoven,"[-0.001220703125, -0.001434326171875, -0.00158...","[[-533.9599211815558, -574.976043514334, -591....","[[-18.31055228501227, -18.31055228501227, -18....","[[1.8854574819499592, 1.8854574819499592, 1.88..."


In [ ]:
# Aggrège les features calculés dans chaque trame pour obtenir des features au niveau de l’extrait
# par calcul de la moyenne des features sur toutes les trames
## MFCC
for i,mfcc in enumerate(df['MFCC']):
  Mfcc_df = pd.DataFrame(mfcc)
  mean = np.array(Mfcc_df.mean(axis =1))
  df['MFCC'][i] = mean
## Delta MFCC
for i,d_mfcc in enumerate(df['Delta MFCC']):
  Delta_mfcc_df = pd.DataFrame(d_mfcc)
  mean = np.array(Delta_mfcc_df.mean(axis =1))
  df['Delta MFCC'][i] = mean
## Delta2 MFCC
for i,d2_mfcc in enumerate(df['Delta2 MFCC']):
  Delta2_mfcc_df = pd.DataFrame(d2_mfcc)
  mean = np.array(Delta2_mfcc_df.mean(axis =1))
  df['Delta2 MFCC'][i] = mean
df

,Compositeurs,Signal numérique,MFCC,Delta MFCC,Delta2 MFCC
0,chopin,"[0.0533447265625, 0.050689697265625, 0.0474853...","[-429.02398859656785, 241.52708224165434, 37.9...","[-1.0500197458467988, -0.5204932719124417, 0.4...","[0.5212377113087039, 0.22007368469946914, -0.1..."
1,chopin,"[-0.051910400390625, -0.05487060546875, -0.057...","[-436.26527157426347, 214.0989497130857, 42.79...","[-0.3298793478308033, -0.32586071207906764, -0...","[0.2224063572094619, 0.07932160286433029, -0.0..."
2,chopin,"[0.0262451171875, 0.025848388671875, 0.0253601...","[-432.91258568866766, 217.98962395092815, 32.3...","[1.2064890355502633, 0.7456222719516481, -0.39...","[-0.22982810725219072, -0.09676664706200232, 0..."
3,chopin,"[0.020904541015625, 0.032623291015625, 0.04464...","[-290.54565671963945, 223.92957024114446, -55....","[0.21329924558032595, 0.27095914372530944, -0....","[0.1696417152675088, -0.013282489460822482, 0...."
4,chopin,"[-0.003143310546875, -0.0030517578125, -0.0028...","[-443.65176239931844, 218.13129969894524, 22.0...","[0.01639739643262425, 0.06158934763908509, -0....","[0.13924353738355388, 0.10531804657128528, -0...."
...,...,...,...,...,...
115,beethoven,"[0.001953125, 0.001953125, 0.00189208984375, 0...","[-670.8083287890472, 151.06517520695763, 65.88...","[0.028660147150013117, -0.058419536125255904, ...","[0.25566551174697033, 0.21566530439467832, -0...."
116,beethoven,"[-0.01007080078125, -0.01324462890625, -0.0163...","[-391.83390434541906, 225.43319088846647, 26.3...","[-0.6814999915978739, -0.39351404089727277, 0....","[0.40219355010106034, 0.30701887843393205, 0.0..."
117,beethoven,"[-0.009490966796875, -0.0093994140625, -0.0092...","[-533.1194049960129, 160.5362435907648, 28.570...","[-0.39512942578550647, -0.5442414357858829, -0...","[0.26984480403832845, 0.23318850717811948, 0.0..."
118,beethoven,"[-0.001220703125, -0.001434326171875, -0.00158...","[-583.7107616453997, 149.15271383319535, 77.62...","[-0.3257123284030883, -0.2909509618933606, 0.0...","[0.2890066269717792, 0.2857817548188808, 0.032..."


In [ ]:
# Calcule le vecteur de descripteurs pour chaque exemple
descripteurs = []
for i,mfcc in enumerate(df['MFCC']):
  delta_mfcc = df['Delta MFCC'][i]
  delta2_mfcc = df['Delta2 MFCC'][i]
  descripteur = np.concatenate((mfcc, delta_mfcc, delta2_mfcc), axis=None)
  descripteurs.append(descripteur)
df.insert(2, 'Descripteur', descripteurs)
df

,Compositeurs,Signal numérique,Descripteur,MFCC,Delta MFCC,Delta2 MFCC
0,chopin,"[0.0533447265625, 0.050689697265625, 0.0474853...","[-429.02398859656785, 241.52708224165434, 37.9...","[-429.02398859656785, 241.52708224165434, 37.9...","[-1.0500197458467988, -0.5204932719124417, 0.4...","[0.5212377113087039, 0.22007368469946914, -0.1..."
1,chopin,"[-0.051910400390625, -0.05487060546875, -0.057...","[-436.26527157426347, 214.0989497130857, 42.79...","[-436.26527157426347, 214.0989497130857, 42.79...","[-0.3298793478308033, -0.32586071207906764, -0...","[0.2224063572094619, 0.07932160286433029, -0.0..."
2,chopin,"[0.0262451171875, 0.025848388671875, 0.0253601...","[-432.91258568866766, 217.98962395092815, 32.3...","[-432.91258568866766, 217.98962395092815, 32.3...","[1.2064890355502633, 0.7456222719516481, -0.39...","[-0.22982810725219072, -0.09676664706200232, 0..."
3,chopin,"[0.020904541015625, 0.032623291015625, 0.04464...","[-290.54565671963945, 223.92957024114446, -55....","[-290.54565671963945, 223.92957024114446, -55....","[0.21329924558032595, 0.27095914372530944, -0....","[0.1696417152675088, -0.013282489460822482, 0...."
4,chopin,"[-0.003143310546875, -0.0030517578125, -0.0028...","[-443.65176239931844, 218.13129969894524, 22.0...","[-443.65176239931844, 218.13129969894524, 22.0...","[0.01639739643262425, 0.06158934763908509, -0....","[0.13924353738355388, 0.10531804657128528, -0...."
...,...,...,...,...,...,...
115,beethoven,"[0.001953125, 0.001953125, 0.00189208984375, 0...","[-670.8083287890472, 151.06517520695763, 65.88...","[-670.8083287890472, 151.06517520695763, 65.88...","[0.028660147150013117, -0.058419536125255904, ...","[0.25566551174697033, 0.21566530439467832, -0...."
116,beethoven,"[-0.01007080078125, -0.01324462890625, -0.0163...","[-391.83390434541906, 225.43319088846647, 26.3...","[-391.83390434541906, 225.43319088846647, 26.3...","[-0.6814999915978739, -0.39351404089727277, 0....","[0.40219355010106034, 0.30701887843393205, 0.0..."
117,beethoven,"[-0.009490966796875, -0.0093994140625, -0.0092...","[-533.1194049960129, 160.5362435907648, 28.570...","[-533.1194049960129, 160.5362435907648, 28.570...","[-0.39512942578550647, -0.5442414357858829, -0...","[0.26984480403832845, 0.23318850717811948, 0.0..."
118,beethoven,"[-0.001220703125, -0.001434326171875, -0.00158...","[-583.7107616453997, 149.15271383319535, 77.62...","[-583.7107616453997, 149.15271383319535, 77.62...","[-0.3257123284030883, -0.2909509618933606, 0.0...","[0.2890066269717792, 0.2857817548188808, 0.032..."


In [ ]:
# Stocke le vecteur dans un fichier contenant la matrice n × d des features de taille d pour les n exemples
matrice = []
for row in df['Descripteur']:
  matrice.append(row)
matrice = np.array(matrice)

matrice_df = pd.DataFrame(matrice)
matrice_df.to_csv('descipteurs.csv', index=False)

##### Construction d'un classifieur qui prédit le compositeur

In [ ]:
# Division en données d'apprentissage et de test
## Descipteurs
descripteurs = pd.read_csv("descipteurs.csv")
X = descripteurs.to_numpy()
## Labels
labels = df['Compositeurs'].to_numpy()
labelEncoder = LabelEncoder()
labelEncoder.fit(labels)
y = labelEncoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Normalisation des vecteurs descripteurs
scaler = StandardScaler(copy=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Création du modèle avec le classifieur "LogisticRegression" de scikit-learn
clf = LogisticRegression()
# Entraînement du modèle
clf.fit(X_train_scaled, y_train)

LogisticRegression()

##### Évaluation des performances du classifieurs

In [ ]:
# Prédictions sur les donnés de test
predictions = clf.predict(X_test_scaled)
predictions

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 3, 1, 2, 1, 1, 1, 2, 3, 3, 1,
       1, 3])

In [ ]:
# Donne la precision du modele
accuracy = clf.score(X_test, y_test)
print("La performance du modèle est de : ", 100*accuracy, " %")

La performance du modèle est de :  62.5  %


##### Exemples mal classés

###### Exhibition

In [ ]:
# Affiche les différences entre prédiction et vrai labels
named_predictions = labelEncoder.inverse_transform(predictions)
named_true_labels = labelEncoder.inverse_transform(y_test)
print("Les prédictions sont : \n-------------------------- \n", named_predictions, "\n\n")
print("Les réels labels étaient : \n-------------------------- \n", named_true_labels)

Les prédictions sont : 
-------------------------- 
 ['chopin' 'beethoven' 'chopin' 'chopin' 'chopin' 'beethoven' 'chopin'
 'chopin' 'chopin' 'beethoven' 'beethoven' 'chopin' 'mozart' 'chopin'
 'liszt' 'chopin' 'chopin' 'chopin' 'liszt' 'mozart' 'mozart' 'chopin'
 'chopin' 'mozart'] 


Les réels labels étaient : 
-------------------------- 
 ['chopin' 'chopin' 'chopin' 'chopin' 'chopin' 'mozart' 'mozart' 'chopin'
 'chopin' 'beethoven' 'chopin' 'mozart' 'chopin' 'chopin' 'liszt' 'liszt'
 'beethoven' 'chopin' 'liszt' 'beethoven' 'mozart' 'chopin' 'chopin'
 'mozart']


In [ ]:
# Affiche les exemples mals classés
print("Les exemples mal-classés dans les données test sont les exemples suivants :")
difference = np.subtract(predictions, y_test)
liste = []
for i,label in enumerate(difference):
  if label != 0:
    liste.append(i+1)
print(liste)

Les exemples mal-classés dans les données test sont les exemples suivants :
[2, 6, 7, 11, 12, 13, 16, 17, 20]


###### Interprétation

In [ ]:
# TODO

### Jeu de mfccs pour la classification de genre (single-label) : MSD-I
Données : 30 712 morceaux de musique </br>
Labels : 15 genres différents (Metal, Rock, etc.)

In [ ]:
# Commande spécifique
%cd /content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi

/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi


#### Descripteurs standards

##### 1. Régression Logistique

###### Extraction des descipteurs

In [ ]:
# Fonction qui charge une matrice de MFCC (n (trames) x d (dimension du descripteur MFCC))
# à partir d'une ligne (entry) du fichier principal : msdi_mapping.csv
def load_mfcc(entry, msdi_path):
    x = np.load(Path(msdi_path) / entry['mfcc'])
    mfcc_matrix = x[entry['msd_track_id']]
    return mfcc_matrix

In [ ]:
# Fonction qui ajoute les ∆MFCC et ∆∆MFCC
def add_deltas(mfcc_matrix):
  trame_number = mfcc_matrix.shape[0]
  descriptor_size = mfcc_matrix.shape[1]
  descriptors_matrix = []
  for mfcc in mfcc_matrix:
    mfcc_delta = delta(mfcc)
    mfcc_delta2 = delta(mfcc, order=2)
    descriptor = np.concatenate((mfcc,mfcc_delta,mfcc_delta2))
    descriptors_matrix.append(descriptor)
  descriptors_matrix = np.array(descriptors_matrix)
  return descriptors_matrix

In [ ]:
# Fonction qui aggrège les caractéristiques calculées dans chaque trame
def aggregate_features(descriptors_matrix):
  sample_descriptor = np.mean(descriptors_matrix, axis=0)
  return sample_descriptor

In [ ]:
# Fonction qui lit chaque fichier son

# TODO : mettre à jour cette fonction pour avoir un train equilibré

def read_files(msdi_path, my_set, samples_number):
  my_data = []
  i = 0
  filter = (mapping_df['set'] == my_set)
  df = mapping_df[filter]
  for index, entry in df.iterrows():
    if i < samples_number:
      genre = entry['genre']
      train_test_set = entry['set']
      mfcc_matrix = load_mfcc(entry, msdi_path)
      descriptors_matrix = add_deltas(mfcc_matrix)
      sample_descriptor = aggregate_features(descriptors_matrix)
      my_data.append(np.array([genre, train_test_set, sample_descriptor], dtype=object))
      i += 1
      if i % 25 == 0:
        print("We finished treating the " + str(i) + "th sample !")
        print("...")
  print("DONE")
  my_data = np.array(my_data)
  return my_data

In [ ]:
# Crée un dataframe du mapping associé à nos données
msdi_mapping_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/msdi_mapping.csv'
mapping_df = pd.read_csv(msdi_mapping_path)
mapping_df.rename(columns={'Unnamed: 0':'Index'}, inplace=True)
mapping_df = mapping_df.set_index('Index')
mapping_df

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
Index,,,,,,,,,
0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/slee...,img/0000848744_200.jpg,mfcc/mfcc_A.npz,0
1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/slee...,img/0000476534_200.jpg,mfcc/mfcc_B.npz,1
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4
...,...,...,...,...,...,...,...,...,...
30708,TRJRMVM128F4259C12,Punk,12066,test,ARC1SV21187FB4CF26,http://artwork-cdn.7static.com/static/img/slee...,img/0000250669_200.jpg,mfcc/mfcc_J.npz,4644
30709,TRJUGOK128F4243DA6,Pop,22364,test,AR5MK521187B98E0B8,http://artwork-cdn.7static.com/static/img/slee...,img/0000139188_200.jpg,mfcc/mfcc_J.npz,4645
30710,TRTMWUS128F425D9AF,Pop,8309,test,ARY2W5X1187FB3CB7A,http://artwork-cdn.7static.com/static/img/slee...,img/0000150164_200.jpg,mfcc/mfcc_T.npz,4646


###### Création des données d'entraînements et de test
Données d'entraînements : 21 383 morceaux </br>
Données de test : 4649 morceaux

In [ ]:
# Fonction qui arrange un dataframe associé à nos données d'entraînement
def create_set_dataframe(my_data):
  df_set = pd.DataFrame(my_data)
  df_set.columns = ["Genre", "Set", "Descripteur"]
  return df_set

In [ ]:
# Fonction qui stocke les vecteurs descripteurs des données d'entraînement et de test
# dans un fichier contenant la matrice m × d des caractéristiques de taille d pour les m morceaux

# TODO : mettre à jour cette fonction pour avoir un stocker au bon endroit

def stock_descriptors(df_set):
  set_name = df_set['Set'][0]
  matrice = []
  for row in df_set['Descripteur']:
    matrice.append(row)
  matrice = np.array(matrice)
  matrice_df = pd.DataFrame(matrice)
  name = 'descripteurs_audio_' + set_name + '.csv'
  matrice_df.to_csv(name, index=False)

In [ ]:
################ NE RELANCER QUE POUR AUGMENTER LE NOMBRE D'EXEMPLES ################

msdi_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/'

# Créer le dataframe de train et le stocke
samples_number_train = 100
train_set = 'train'
my_data_train = read_files(msdi_path, train_set, samples_number_train)
df_train = create_set_dataframe(my_data_train)
stock_descriptors(df_train)

# Créer le dataframe de test et le stocke
samples_number_test = 10
test_set = 'test'
my_data_test = read_files(msdi_path, test_set, samples_number_test)
df_test = create_set_dataframe(my_data_test)
stock_descriptors(df_test)

KeyboardInterrupt: ignored

###### Construction d'un classifieur qui prédit le genre

In [ ]:
# Chargement des données d'apprentissage et de test
## Descripteurs
descripteurs_train = pd.read_csv("descripteurs_audio_train.csv")
X_train = descripteurs_train.to_numpy()

descripteurs_test = pd.read_csv("descripteurs_audio_test.csv")
X_test = descripteurs_test.to_numpy()

## Labels
labels_train = df_train['Genre'].iloc[:1000].to_numpy()
labels_test = df_test['Genre'].iloc[:100].to_numpy()
labelEncoder = LabelEncoder()
y_train = labelEncoder.fit_transform(labels_train)
y_test = labelEncoder.transform(labels_test)

In [ ]:
# Normalisation des vecteurs descripteurs
scaler = StandardScaler(copy=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Création du modèle avec le classifieur "LogisticRegression" de scikit-learn
clf = LogisticRegression(max_iter=500)
# Entraînement du modèle
clf.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=500)

###### Évaluation des performances du classifieurs

In [ ]:
# Prédictions sur les donnés de test
predictions = clf.predict(X_test_scaled)
predictions

array([12, 12,  2,  1,  6,  4, 12, 12, 12, 12,  2, 12, 12, 12,  6, 12, 12,
       12,  6,  7, 12, 12, 12, 12, 12, 12,  2, 12, 12, 10,  7,  6,  9,  6,
        9, 12, 12, 12, 12, 12, 12,  4, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12,  7, 12, 12, 12, 12, 12, 12, 12, 10, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 10, 12, 12,  4,
       12, 12, 12, 12, 10,  0,  9, 12, 12, 12, 12, 12,  6, 12, 10])

In [ ]:
# Donne la precision du modele
accuracy = clf.score(X_test, y_test)
print("La performance du modèle est de : ", 100*accuracy, " %")

La performance du modèle est de :  55.00000000000001  %


###### Exemples mal classés

*Exhibition*

In [ ]:
# Affiche les différences entre prédiction et vrai labels
named_predictions = labelEncoder.inverse_transform(predictions)
named_true_labels = labelEncoder.inverse_transform(y_test)
print("Les prédictions sont : \n-------------------------- \n", named_predictions, "\n\n")
print("Les réels labels étaient : \n-------------------------- \n", named_true_labels)

Les prédictions sont : 
-------------------------- 
 ['Rock' 'Rock' 'Electronic' 'Country' 'Metal' 'Jazz' 'Rock' 'Rock' 'Rock'
 'Rock' 'Electronic' 'Rock' 'Rock' 'Rock' 'Metal' 'Rock' 'Rock' 'Rock'
 'Metal' 'Pop' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Electronic'
 'Rock' 'Rock' 'Reggae' 'Pop' 'Metal' 'Rap' 'Metal' 'Rap' 'Rock' 'Rock'
 'Rock' 'Rock' 'Rock' 'Rock' 'Jazz' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock'
 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Pop' 'Rock' 'Rock'
 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Reggae' 'Rock' 'Rock' 'Rock' 'Rock'
 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock'
 'Rock' 'Rock' 'Rock' 'Rock' 'Reggae' 'Rock' 'Rock' 'Jazz' 'Rock' 'Rock'
 'Rock' 'Rock' 'Reggae' 'Blues' 'Rap' 'Rock' 'Rock' 'Rock' 'Rock' 'Rock'
 'Metal' 'Rock' 'Reggae'] 


Les réels labels étaient : 
-------------------------- 
 ['Electronic' 'Pop' 'Electronic' 'Latin' 'Rock' 'Pop' 'Rock' 'Rock' 'Rock'
 'Rock' 'Electronic' 'Rock' 'Rock' 'Rock' 'Metal' 'Blues' 'Rock' 'Ro

In [ ]:
# Affiche les exemples mals classés
print("Les exemples mal-classés dans les données test sont les exemples suivants :")
difference = np.subtract(predictions, y_test)
liste = []
for i,label in enumerate(difference):
  if label != 0:
    liste.append(i+1)
print(liste)

Les exemples mal-classés dans les données test sont les exemples suivants :
[1, 2, 4, 5, 6, 16, 19, 20, 23, 25, 27, 35, 37, 38, 39, 40, 41, 42, 45, 47, 50, 51, 55, 59, 61, 63, 64, 70, 71, 72, 75, 76, 78, 79, 80, 81, 82, 84, 85, 90, 91, 92, 96, 98, 100]


*Interprétation* </br>
En regardant de plus près, on constate que la plupart des exemples mal classés ont été prédits du genre "rock" à tort. Cela vient d'un déséquilibre présent dans les données d'entraînement. Nous verrons plus tard si l'apprentissage multimodal permet de résoudre ce problème ou bien si nous devrons trouver une autre solution.

##### 2. Système deep & CNN

###### Enregistrement des matrices de descripteurs

**Résultat empirique** </br>
Dans le jeu de données d'entraînement fourni, il y a un déséquilibre des classes qui influe sur l'entraînement. La classe la plus présente est la classe rock et la moins présente est présente en 329 exemplaires (cf. histogramme sur le notebook relatif aux pochettes d'album). On construit donc un nouveau jeu de données d'entraînement qui prend en compte ces informations.

In [ ]:
# Fonction qui lit le fichier mapping pour le set donné et stocke pour un certain nombre de morceaux (samples_number)
# la matrice (n * d) des descripteur de taille d sur les n trame en s'assurant qu'il y en ai le même nombre par genre (class_samples_max_number)
def read_files_deep(msdi_path, my_set, samples_number, class_samples_max_number):
  labels = []
  matrices = []
  i = 0
  N_0 = 500
  class_number = 15
  filter = (mapping_df['set'] == my_set)
  df = mapping_df[filter]
  counts = np.zeros(class_number)
  for index, entry in df.iterrows():
    genre = entry['genre']
    labelId = labelEncoder.transform([genre])[0]
    # Vérifie l'équilibre des classes et la taille du jeu de donnée
    if i < samples_number:
      if counts[labelId] != class_samples_max_number:
        mfcc_matrix = load_mfcc(entry, msdi_path)
        # Tronque à 500 trames
        if mfcc_matrix.shape[0] > 500:
          to_delete = mfcc_matrix.shape[0] - N_0
          mfcc_matrix = mfcc_matrix[:-to_delete, :]
          descriptors_matrix = add_deltas(mfcc_matrix)
          stock_descriptors(entry, descriptors_matrix)
          labels.append(genre)
          matrices.append(descriptors_matrix)
          i += 1
          counts[labelId] += 1
          if i % 25 == 0:
            print("We finished treating the " + str(i) + "th sample !")
            print("...")
  print("DONE")
  labels = np.array(labels)
  matrices = np.array(matrices)
  return labels, matrices

In [ ]:
# Fonction qui stocke la matrices des descripteurs de chaque trame pour un exemple
# dans un fichier portant le même nom
def stock_descriptors(entry, descriptors_matrix):
  set_name = entry['set'] + '_'
  id = entry['msd_track_id']
  matrice_df = pd.DataFrame(descriptors_matrix)
  if entry['set'] == 'train':
    directory_path = 'out_audio/out_train/'
  elif entry['set'] == 'test':
    directory_path = 'out_audio/out_test/'
  name = directory_path + 'descripteurs_audio_CNN_' + set_name + id + '.csv'
  matrice_df.to_csv(name, index=False)

In [ ]:
# Encode les étiquettes
genres = []
df_genres = pd.read_csv('labels.csv', header=None)
for index, genre in df_genres.iterrows():
  genres.append(genre[0])

labelEncoder = LabelEncoder()
genres = labelEncoder.fit_transform(genres)

In [ ]:
################ NE RELANCER QUE POUR AUGMENTER LE NOMBRE D'EXEMPLES ################
# Créer les arrays des étiquettes et matrices d'apprentissage et les stocke
msdi_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/'
samples_number_train = 4875 # le max qu'on puisse mettre est 329 * 15 = 4 935
train_set = 'train'
class_samples_max_number = 325 # le max qu'on puisse mettre est 329
etiquettes_train, matrices_train = read_files_deep(msdi_path, train_set, samples_number_train, class_samples_max_number)

We finished treating the 25th sample !
...
We finished treating the 50th sample !
...
We finished treating the 75th sample !
...
We finished treating the 100th sample !
...
We finished treating the 125th sample !
...
We finished treating the 150th sample !
...
We finished treating the 175th sample !
...
We finished treating the 200th sample !
...
We finished treating the 225th sample !
...
We finished treating the 250th sample !
...
We finished treating the 275th sample !
...
We finished treating the 300th sample !
...
We finished treating the 325th sample !
...
We finished treating the 350th sample !
...
We finished treating the 375th sample !
...
We finished treating the 400th sample !
...
We finished treating the 425th sample !
...
We finished treating the 450th sample !
...
We finished treating the 475th sample !
...
We finished treating the 500th sample !
...
We finished treating the 525th sample !
...
We finished treating the 550th sample !
...
We finished treating the 575th samp

In [ ]:
################ NE RELANCER QUE POUR AUGMENTER LE NOMBRE D'EXEMPLES ################
# Créer les arrays des étiquettes et matrices de test et les stocke
msdi_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/'
samples_number_test = 675 # le max qu'on puisse mettre est 45 * 15 = 675
test_set = 'test'
class_samples_max_number = 45 # le max qu'on puisse mettre est 45
etiquettes_test, matrices_test = read_files_deep(msdi_path, test_set, samples_number_test, class_samples_max_number)

We finished treating the 25th sample !
...
We finished treating the 50th sample !
...
We finished treating the 75th sample !
...
We finished treating the 100th sample !
...
We finished treating the 125th sample !
...
We finished treating the 150th sample !
...
We finished treating the 175th sample !
...
We finished treating the 200th sample !
...
We finished treating the 225th sample !
...
We finished treating the 250th sample !
...
We finished treating the 275th sample !
...
We finished treating the 300th sample !
...
We finished treating the 325th sample !
...
We finished treating the 350th sample !
...
We finished treating the 375th sample !
...
We finished treating the 400th sample !
...
We finished treating the 425th sample !
...
We finished treating the 450th sample !
...
We finished treating the 475th sample !
...
We finished treating the 500th sample !
...
We finished treating the 525th sample !
...
We finished treating the 550th sample !
...
We finished treating the 575th samp

In [ ]:
# Utilise glob pour obtenir tous les fichiers csv du dossier
def recupere_matrice(chemin):
  fichiers_csv = glob.glob(os.path.join(chemin, "*.csv"))
  matrices = []
  # Boucle sur la liste des fichiers csv
  for fichier in fichiers_csv:
    # Note la matrice de descripteurs par trame du csv dans une matrice numpy
    matrice = np.loadtxt(my_path, delimiter=",",  skiprows=1)
    matrices.append(matrice)
  matrices = np.array(matrices)
  return(matrices)

In [ ]:
chemin_train = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/out_audio/out_train/'
matrices_train = recupere_matrice(chemin_train) #1 minute 40 secondes

In [ ]:
chemin_test = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/out_audio/out_test/'
matrices_test = recupere_matrice(chemin_test) # 15 secondes

In [ ]:
N_0, dimension_descriptors = 500, 36 # Taille des "images"
nombre_genres = 15
nombre_exemples_appr = len(matrices_train)
nombre_exemples_test = len(matrices_test)

# Reforme en (nombre_exemple_set, 500, 36, 1)
x_train = matrices.reshape(matrices.shape[0], N_0, dimension_descriptors, 1)
x_test = matrices_test.reshape(matrices_test.shape[0], N_0, dimension_descriptors, 1)

# Révise le type des éléments
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
# Encodage sous forme d'entiers pour les étiquettes
y_train = labelEncoder.transform(etiquettes_train)
y_test = labelEncoder.transform(etiquettes_test)

# Encodage d'entier à one-hot
y_train = keras.utils.np_utils.to_categorical(y_train, nombre_genres)
y_test = keras.utils.np_utils.to_categorical(y_test, nombre_genres)

###### CNN

In [ ]:
# Création du réseau de neurone convolutionnel (CNN)
dimension_entree = (N_0, dimension_descriptors, 1)

CCNmodel = Sequential()
CCNmodel.add(Conv2D(32, (4, 3), activation='relu', strides=(1, 1), padding='same', input_shape=dimension_entree))
CCNmodel.add(MaxPooling2D((4, 2)))
CCNmodel.add(Conv2D(64, (4, 3), activation='relu', strides=(1, 1), padding='same', input_shape=dimension_entree))
CCNmodel.add(MaxPooling2D((4, 2)))
CCNmodel.add(Conv2D(64, (4, 3), activation='relu', strides=(1, 1), padding='same', input_shape=dimension_entree))
CCNmodel.add(MaxPooling2D((4, 1)))
CCNmodel.add(Conv2D(32, (1, 1), activation='relu', strides=(1, 1), padding='same', input_shape=dimension_entree))
CCNmodel.add(MaxPooling2D((4, 3)))
CCNmodel.add(Flatten())
CCNmodel.add(Dense(nombre_genres, activation='sigmoid'))

In [ ]:
# Hyperparamètres
batch_size = 32
epochs = 100
loss = 'categorical_crossentropy'
optimizer='adam'
metrics='accuracy'

In [ ]:
CCNmodel.compile(loss=loss, optimizer=optimizer, metrics=metrics)
CCNmodel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 500, 36, 32)       416       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 125, 18, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 125, 18, 64)       24640     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 31, 9, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 31, 9, 64)         49216     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 7, 9, 64)        

In [ ]:
CNNclf = CCNmodel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 10.9896 - accuracy: 0.0667
Epoch 2/100
1/1 [==============================] - 0s 284ms/step - loss: 5.4608 - accuracy: 0.0667
Epoch 3/100
1/1 [==============================] - 0s 311ms/step - loss: 4.6488 - accuracy: 0.0667
Epoch 4/100
1/1 [==============================] - 0s 281ms/step - loss: 3.5716 - accuracy: 0.0667
Epoch 5/100
1/1 [==============================] - 0s 282ms/step - loss: 3.4422 - accuracy: 0.0667
Epoch 6/100
1/1 [==============================] - 0s 277ms/step - loss: 3.4351 - accuracy: 0.0667
Epoch 7/100
1/1 [==============================] - 0s 287ms/step - loss: 3.0627 - accuracy: 0.0667
Epoch 8/100
1/1 [==============================] - 0s 280ms/step - loss: 3.0750 - accuracy: 0.0667
Epoch 9/100
1/1 [==============================] - 0s 283ms/step - loss: 2.9593 - accuracy: 0.0667
Epoch 10/100
1/1 [==============================] - 0s 293ms/step - loss: 3.0203 - accuracy: 0.0667
Epoch 11/10

In [ ]:
def evaluate(x_test, y_test):
  y_pred = CCNmodel.predict(x_test)
  labels = genres
  ## ATTENTION peut être faut-il ici prendre le softmax de y_pred pour le comparer au y_test
  my_roc_score = roc_auc_score(y_test, y_pred, labels=labels)
  return my_roc_score, y_pred

In [ ]:
score, y_pred = evaluate(x_test, y_test)
y_pred[0]

array([0.5842107 , 0.5868314 , 0.5868381 , 0.58522034, 0.58656937,
       0.5875752 , 0.58475876, 0.5862318 , 0.5856259 , 0.58638096,
       0.58798033, 0.5860157 , 0.58721155, 0.5854015 , 0.5868663 ],
      dtype=float32)

#### Apprentissage de représentations

##### Chargement des représentations

In [ ]:
# Fonction pour charger la liste des labels de genre
def get_label_list(msdi_path):
    df = pd.read_csv(Path(msdi_path) / 'labels.csv', header=None)
    return list(df.iloc[:, 0])

In [ ]:
msdi_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/'

# Encode les genres
labels = get_label_list(msdi_path)

labelEncoder = LabelEncoder()
labels = labelEncoder.fit_transform(labels)

In [ ]:
msdi_mapping_path = msdi_path + 'msdi_mapping.csv'

# Crée un dataframe du mapping associé à nos données
mapping_df = pd.read_csv(msdi_mapping_path)
mapping_df.rename(columns={'Unnamed: 0':'Index'}, inplace=True)
mapping_df = mapping_df.set_index('Index')
mapping_df.head(5)

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
Index,,,,,,,,,
0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/slee...,img/0000848744_200.jpg,mfcc/mfcc_A.npz,0
1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/slee...,img/0000476534_200.jpg,mfcc/mfcc_B.npz,1
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4


In [ ]:
# Crée un dataframe associé aux données d'entraînement
train_filter = (mapping_df['set'] == 'train')
train_df = mapping_df[train_filter]

# Crée un dataframe associé aux données de test
test_filter = (mapping_df['set'] == 'test')
test_df = mapping_df[test_filter]
test_df = test_df.reset_index()

In [ ]:
train_df.groupby(['genre']).count()['msd_track_id'].mean()

1425.5333333333333

Le jeu de données d'entraînement est désequilibré, il contient, pour certains genres comme le *'Rock'* plus de 3000 morceaux contre 156 exemples pour le *'New Age'*. On souhaite le ré-ajuster. On pourrait choisir de ne garder qu'un même nombre de morceau par genre, mais cela rendrait le jeu d'entraînement trop petit, c'est pourquoi quitte à conserver un certain désequilibre, on préfère l'étoffer et garder une limite à **1425** morceaux par genre. (C'est le nombre moyen de morceau par genre, tous genres confondus !)

In [ ]:
# Fonction qui réajuste le jeu de donnnées d'entraînement
def reajust_train(train_df):
  class_number = 15
  class_samples_max_number = 1425
  counts = np.zeros(class_number)
  for i in train_df.index:
    label = train_df['genre'][i]
    labelId = labelEncoder.transform([label])[0]
    if counts[labelId] >= class_samples_max_number:
      train_df = train_df.drop(i)
    else :
      counts[labelId] +=1
  return train_df

In [ ]:
reajusted_train_df = reajust_train(train_df)
reajusted_train_df = reajusted_train_df.reset_index()
reajusted_train_df.head(5)

,Index,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
0,0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/slee...,img/0000848744_200.jpg,mfcc/mfcc_A.npz,0
1,1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/slee...,img/0000476534_200.jpg,mfcc/mfcc_B.npz,1
2,2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
3,3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
4,4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4


In [ ]:
# Fonction pour charger un vecteur de descripteurs à partir d'une ligne `entry` du fichier principal
def load_deep_audio_features(entry, msdi_path):
    subset_file = 'X_{}_audio_MSD-I.npy'.format(entry['set'])
    x = np.load(Path(msdi_path) / 'deep_features' / subset_file, mmap_mode='r')
    idx = entry['deep_features']
    return x[idx, :]

In [ ]:
# Crée les données d'entraînement et de test
nombre_morceaux_train = reajusted_train_df.shape[0]
dimension_descripteur = 2048
nombre_genres = 15
X_train = np.empty((nombre_morceaux_train, dimension_descripteur))
y_train = np.empty((nombre_morceaux_train, nombre_genres))

nombre_morceaux_test = test_df.shape[0]
X_test = np.empty((nombre_morceaux_test, dimension_descripteur))
y_test = np.empty((nombre_morceaux_test, nombre_genres))

for index, entry in reajusted_train_df.iterrows():
  descripteur = load_deep_audio_features(entry, msdi_path)
  X_train[index] = descripteur
  # genre -> entier
  y_integer = labelEncoder.transform([entry['genre']])[0]
  # entier -> one-hot
  y_one_hot = keras.utils.np_utils.to_categorical(y_integer, nombre_genres)
  y_train[index] = y_one_hot

for index, entry in test_df.iterrows():
  descripteur = load_deep_audio_features(entry, msdi_path)
  X_test[index] = descripteur
  # genre -> entier
  y_test_integer = labelEncoder.transform([entry['genre']])[0]
  # entier -> one-hot
  y_test_one_hot = keras.utils.np_utils.to_categorical(y_test_integer, nombre_genres)
  y_test[index] = y_test_one_hot

In [ ]:
y_train.shape

(15075, 15)

##### Classifieur

Classifieur perceptron multicouche avec une couche de sortie sigmoïde

In [ ]:
# Création du modèle avec le classifieur "MLPClassifier" de scikit-learn
clf = MLPClassifier(random_state=1, max_iter=300, activation='logistic') # logistic = sigmoide

In [ ]:
# Entraînement du modèle ≃ 7 minutes 30
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(activation='logistic', max_iter=300, random_state=1)

In [ ]:
# Prédictions sur les donnés de test
y_pred = clf.predict_proba(X_test)
y_pred

array([[8.47992403e-06, 8.93264175e-02, 3.78565034e-01, ...,
        7.11364158e-02, 2.16016913e-02, 1.20062140e-02],
       [3.36959844e-05, 8.22425997e-02, 8.83879162e-03, ...,
        6.66537235e-03, 9.46914127e-04, 6.94193763e-05],
       [2.15000135e-05, 4.97907336e-05, 2.58083623e-02, ...,
        5.57469884e-01, 5.88800239e-03, 3.89026661e-05],
       ...,
       [1.12348653e-04, 5.89751702e-01, 2.98893618e-06, ...,
        1.11723862e-02, 4.14020548e-02, 3.73554802e-03],
       [1.60234607e-01, 4.89542322e-05, 9.40122775e-03, ...,
        5.11000806e-01, 3.16017347e-04, 1.00105572e-05],
       [1.14114211e-04, 1.91675002e-05, 2.41540488e-01, ...,
        4.94671312e-03, 3.94307336e-02, 6.37976705e-07]])

In [ ]:
# Donne la precision du modele
accuracy = clf.score(X_test, y_test)
print("La précision du modèle est de : ", 100*accuracy, " %")

my_roc_score = roc_auc_score(y_test, y_pred)
print("Le ROC score du modèle est de : ", 100*my_roc_score, " %")

La précision du modèle est de :  31.06044310604431  %
Le ROC score du modèle est de :  82.89514039600549  %


156 exemples par classes fournit une précision du modèle de 8 % et un ROC score de 56 %. </br> Pour un nombre maximal de 1425 exemples par genre, la précision est de 31 % et le ROC score de 83 %.


In [ ]:
import pickle

# Fonction qui permet de sauvegarder le modèle
def save_model(model,model_path):
    pickle.dump(model, open(model_path, "wb"))

model_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/out_audio/audio_model'
save_model(clf, model_path)